In [1]:
import requests
import json
import urllib
from collections import defaultdict
from operator import itemgetter
from haversine import Unit
import haversine as hs
import time

In [2]:
import os

home_relative_dir = '..'

os.chdir(home_relative_dir)
os.getcwd()

'/Users/adityachawla/skunkworks22/RightRoute'

In [3]:
with open('apikey.txt','r') as f:
    api_key = f.read().strip()

In [4]:
# routes
r = requests.get(f'http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json',params=[('key',api_key)])
all_routes = json.loads(r.content)

In [5]:
def get_vehicle_data(api_key):
    r = requests.get('http://bustime.mta.info/api/siri/vehicle-monitoring.json',params=[('key',api_key),('version',2)])
    if r.ok:
        return json.loads(r.content)['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
    print('Vehicle GET call failed')
    return None

def calculate_distance(loc1,loc2,unit = Unit.MILES):
    return hs.haversine((loc1['Latitude'],loc1['Longitude']),(loc2['Latitude'],loc2['Longitude']),unit=unit)

def extract_groups(vehicle_activities):
    groups = defaultdict(lambda : defaultdict(list))
    for vehicle_activity in vehicle_activities:
        vehicle_activity = vehicle_activity['MonitoredVehicleJourney']
        line_ref,direction_ref, location,v_id = itemgetter('LineRef', 'DirectionRef','VehicleLocation','VehicleRef')(vehicle_activity)
        groups[line_ref][direction_ref].append({
            'vehicle_id':v_id,
            'location':location
        })
    return groups


In [22]:
vehicle_activities = get_vehicle_data(api_key)
groups = extract_groups(vehicle_activities)

In [8]:
THRESH = 0.3
latest_update = []
for route,route_vehicles in groups.items():
    for route_dir,dir_vehicles in route_vehicles.items():
        l = len(dir_vehicles)
        for i in range(l):
            for j in range(i+1,l):
                distance = calculate_distance(dir_vehicles[i]['location'],dir_vehicles[j]['location'])
                if distance<THRESH:
                    latest_update.append({
                        'bus1':dir_vehicles[i]['vehicle_id'],
                        'bus2':dir_vehicles[j]['vehicle_id'],
                        'distance': round(distance,3)
                    })

In [9]:
path_to_service_acc = '/Users/adityachawla/skunkworks22/RightRoute/firestore-access-key.json'
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials
cred = credentials.ApplicationDefault()
project_id = 'aditya-gcp-342814'
cred = credentials.Certificate(path_to_service_acc)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [10]:
doc_ref = db.collection(u'skunkworks_22').document(u'bus_tracking')

In [16]:
doc = doc_ref.get()

In [19]:
doc.get('current')

True

In [11]:
old_data_ref = db.collection(u'skunkworks_22').document(u'old_bus_data')

In [13]:
doc_ref.update({
    'current':latest_update
})

update_time {
  seconds: 1649952866
  nanos: 589604000
}

In [33]:
# old_doc_ref = doc_ref.collection('old')
# old_doc_ref.set({
#     'something1':latest_update
# })

In [63]:
import os
import requests
import json
import urllib
from collections import defaultdict
from operator import itemgetter
from haversine import Unit
import haversine as hs
from google.cloud import firestore
import time

THRESH = 0.3
with open('apikey.txt','r') as f:
    api_key = f.read().strip()
db = firestore.Client.from_service_account_json('firestore-access-key.json')
doc_ref = db.collection(u'skunkworks_22').document(u'bus_tracking')

def get_vehicle_data(api_key):
    r = requests.get('http://bustime.mta.info/api/siri/vehicle-monitoring.json',params=[('key',api_key),('version',2)])
    if r.ok:
        return json.loads(r.content)['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
    print('Vehicle GET call failed')
    return None

def calculate_distance(loc1,loc2,unit = Unit.MILES):
    return hs.haversine((loc1['Latitude'],loc1['Longitude']),(loc2['Latitude'],loc2['Longitude']),unit=unit)

def extract_groups(vehicle_activities):
    groups = defaultdict(lambda : defaultdict(list))
    for vehicle_activity in vehicle_activities:
        vehicle_activity = vehicle_activity['MonitoredVehicleJourney']
        line_ref,direction_ref, location,v_id = itemgetter('LineRef', 'DirectionRef','VehicleLocation','VehicleRef')(vehicle_activity)
        groups[line_ref][direction_ref].append({
            'vehicle_id':v_id,
            'location':location,
            
        })
    return groups

def extract_latest_mta_update(groups):
    latest_update = []
    for route,route_vehicles in groups.items():
        for route_dir,dir_vehicles in route_vehicles.items():
            l = len(dir_vehicles)
            for i in range(l):
                for j in range(i+1,l):
                    distance = calculate_distance(dir_vehicles[i]['location'],dir_vehicles[j]['location'])
                    if distance<THRESH:
                        latest_update.append({
                            'bus1':dir_vehicles[i]['vehicle_id'],
                            'bus2':dir_vehicles[j]['vehicle_id'],
                            'distance': round(distance,3),
                            'route':route,
                            'route_dir':int(route_dir),
                            'highlight': random.randint(0,1)
                        })
    return latest_update

def run_update_task():
    vehicle_activities = get_vehicle_data(api_key)
    groups = extract_groups(vehicle_activities)
    latest_update = extract_latest_mta_update(groups)
    doc_ref.update({
        'current':latest_update
    })

def run_mta_job(event, context):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    for _ in range(3):
        run_update_task()
    
    
    print('DONE') 
    

In [64]:
extract_latest_mta_update(groups)

[{'bus1': 'MTA NYCT_9769',
  'bus2': 'MTA NYCT_9834',
  'distance': 0.014,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9769',
  'bus2': 'MTA NYCT_9597',
  'distance': 0.019,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9756',
  'bus2': 'MTA NYCT_4257',
  'distance': 0.156,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9754',
  'bus2': 'MTA NYCT_4346',
  'distance': 0.11,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9834',
  'bus2': 'MTA NYCT_9597',
  'distance': 0.005,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9846',
  'bus2': 'MTA NYCT_3954',
  'distance': 0.034,
  'route': 'MTA NYCT_M4',
  'route_dir': 0},
 {'bus1': 'MTA NYCT_9770',
  'bus2': 'MTA NYCT_4350',
  'distance': 0.064,
  'route': 'MTA NYCT_M4',
  'route_dir': 1},
 {'bus1': 'MTA NYCT_9779',
  'bus2': 'MTA NYCT_9762',
  'distance': 0.152,
  'route': 'MTA NYCT_M4',
  'route_dir': 1},
 {'bus1': 'MTA NYCT_4072',
  'bus2': 'MTA

In [59]:
import random
random.randint(0,1)

1